## Calculate labelling rates

This notebook provides data and functionality for tracking number of assignments mapped over time, average labelling rates, assignments completed per day, etc.  



## Set-up 

In [ ]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('src'))
sys.path.insert(0, module_path)

from labelreview import labelReview

### Query data

In [ ]:
lr = labelReview(config="config-db.yaml")
query = \
    "SELECT name,hit_id,assignment_id,worker_id,email,"\
    "start_time,completion_time,score,status,kml_type,mapped_count "\
    "FROM assignment_data "\
    "LEFT JOIN hit_data USING (hit_id) "\
    "LEFT JOIN kml_data USING (name) "\
    "LEFT JOIN users ON assignment_data.worker_id = users.id"
assignments = lr.get_data(query)

### Complile daily rate function

In [ ]:
def daily_rates(assignments, plot=True, date_range=None, workers=None):
    """Calculates the daily rate and days worked for one or more labellers 
    for a given date range, taking full date range and labellers and plotting 
    results by default

    Params
    -------
    assignments : DataFrame
        Results of query into database
    plot : bool
        To plot results or return summary DataFrame. Defaults to plot
    date_range : list
        An optional list of dates defining a range to query, e.g. 
        ["2024-01-01", "2024-01-14"]
    workers : list
        A list of specific worker ids, if results from just one worker are 
        needed

    Returns
    -------
    Plot or DataFrame
    """
    
    if not workers:
        workers = assignments.worker_id.unique()
    if not date_range:
        dates = [
            pd.to_datetime(assignments.completion_date).min().date(), 
            pd.to_datetime(assignments.completion_date).max().date()
        ]
    else:
        assert isinstance(date_range, list), "need list of date strings"
        dates = [pd.to_datetime(date).date() for date in date_range]
    
    rates = (
        assignments
        .value_counts(subset=["worker_id", "completion_date"])
        .reset_index()
        .query("worker_id in @workers")
        .query("completion_date >= @dates[0] & completion_date <= @dates[1]")
        .groupby("worker_id")
        .agg({'completion_date': 'size', 'count':'mean'})
        .rename(columns={"completion_date": "days_worked", "count": "average_rate"})
        .reset_index()
    )
    if plot:
        rates.plot(x="worker_id", kind="bar", figsize=(14,5), 
                   ylabel="Days worked/assignments completed")
    else:
        rates    

## Analyze assignment times per labeller

### Calculate times per assignment

Note: for averages we drop abandoned assignments and ones that took longer than 2 hours, as these likely represented sites that were not mapped in one sitting. 

In [ ]:
assignments["tot_time"] = assignments.apply(
    lambda x: (x.completion_time-x.start_time).total_seconds() / (60 * 60), 
    axis=1
)
assignments["completion_date"] = assignments.completion_time.dt.date

mean_times = (
    assignments
    .query("status != 'Abandoned' & tot_time<2")
    .value_counts(subset=["worker_id", "kml_type", "tot_time"])
    .reset_index()
    .groupby(["worker_id", "kml_type"])["tot_time"]
    .mean()
    .apply(lambda x: x * 60)
    .reset_index()
    .rename(columns={"tot_time": "avg_time"})
)

### Total time per assignment

In [ ]:
(
    assignments
    .query("status != 'Abandoned' & tot_time<2")
    .get("tot_time")
    .plot(kind="hist", title="Total time", xlabel="Hours")
)
None

### Average time per assignment

According to different assignment types

In [ ]:
(
    mean_times
    .pivot_table(values="avg_time", index="worker_id", columns="kml_type")
    .plot(kind="bar", ylabel="Minutes", figsize=(14,5),
          title="Average time to label one site")
)
None

In [ ]:
overall_mean = mean_times.avg_time.mean().round(2)
print(f"Overall average time per assignment: {overall_mean.round(2)} minutes")
print(f"Expected assignments over 7 hrs: {((60 * 7) / overall_mean).round()}")
print(f"Expected assignments over 8 hrs: {((60 * 8) / overall_mean).round()}")

Average time to label F sites

## Sites per day

### Total sites per day

In [ ]:
(
    assignments
    .value_counts(subset=["completion_date", "kml_type"])
    .unstack(level=1)
    .reset_index()
    .plot(x="completion_date", kind="bar", subplots=True, figsize=(14,5))   
)
None

### Average daily rate per worker

Run the `daily_rates` function to calculate average number of sites per worker in a given time period, giving both the total days worked during the time period and the average assignments completed. Examples of usage are provided below:

1. For all workers across the full project period to date

In [ ]:
daily_rates(assignments)

2. All workers in a given time period

In [ ]:
daily_rates(assignments, date_range=["2024-01-01", "2024-01-21"])

3. Specific workers in a specific date range

In [ ]:
daily_rates(assignments, workers=[10, 22, 27], date_range=["2024-01-01", "2024-01-21"])

### Other queries

Custom queries can be written, for example total assignments per day for a specific worker for a specific assignment type

In [ ]:
(
    assignments
    .query("worker_id==27 & kml_type=='F'")
    .groupby(["completion_date"])["completion_date"]
    .count()
    .plot(kind="bar", figsize=(14,5))
)
None